<p style="font-family: Arial; font-size:3.75em;color:purple; font-style:bold"><br>
'NEXT FLEX' Kripton Maps</p><br>

This notebook makes Kripton Maps fow two different energy readouts: PMTs and FIBERs

Its main porpouse is to check ...


In [ ]:
from IPython.core.display import HTML
css = open('css/style-table.css').read() + open('css/style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [ ]:
# General importings
import os
import sys
import logging
import math

import glob

import numpy  as np
import tables as tb
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors

# Specific IC stuff
import invisible_cities.core.system_of_units  as units

from invisible_cities.io.mcinfo_io import load_mcparticles_df
from invisible_cities.io.mcinfo_io import load_mchits_df
from invisible_cities.io.mcinfo_io import load_mcsensor_response_df
from invisible_cities.io.mcinfo_io import get_sensor_binning
from invisible_cities.io.mcinfo_io import get_sensor_types

from my_IC.mcinfo_io import get_sensor_positions


### In case of NOT having IC, use these alternative imports ###
#import my_IC.system_of_units  as units
#
#from my_IC.mcinfo_io import load_mcparticles_df
#from my_IC.mcinfo_io import load_mchits_df
#from my_IC.mcinfo_io import load_mcsensor_response_df
#from my_IC.mcinfo_io import get_sensor_binning
#from my_IC.mcinfo_io import get_sensor_types
#from my_IC.mcinfo_io import get_sensor_positions

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
plt.rcParams["figure.figsize"] = 8, 6
plt.rcParams["font.size"     ] = 14

In [ ]:
VERBOSITY = True

# INPUT DATA

Input data consist of 1e4 Kripton events generated randomly in ACTIVE, stored in 100 different files.
Different events have different event_ids despite the file they are stored in.

In order to make a fair comparison, Next-Flex had both energy-reading sensor types:
* PMTs, wich have the real PDE implemented in the simulations so their response are 'pes'.
* FIBER_SENSORs, with a PDE=1 in the sims, so their response are 'photons'.

As we have both energy-reading systems implemented in the simulation, the light they see is slightly lower than if we had implemented just one of them.

The data must be placed locally in a a given PATH.

Data files can be found in:
* NEUTRINOS: '/data4/NEXT/users/jmunoz/Development/NextFlex/data/Kr83/ACTIVE/dst'
* MAJORANA:  '/home/jmunoz/Development/NextFlex/data/Kr83/ACTIVE/dst'

In [ ]:
LOCAL_PATH = "./data/kripton/"

iFileNames = glob.glob(f"{LOCAL_PATH}*.h5")

if VERBOSITY:
    print(f"{len(iFileNames)} input file names ...\n")
    #for ifname in iFileNames: print(ifname)

# Reading mc info

Using the first iFile to get the general Info

In [ ]:
iFileName = iFileNames[0]

In [ ]:
mcConfig = pd.read_hdf(iFileName, 'MC/configuration')
mcConfig.set_index("param_key", inplace = True)
mcConfig

In [ ]:
mcSensorBinnings = get_sensor_binning(iFileName)
mcSensorBinnings

In [ ]:
mcSensorTypes = get_sensor_types(iFileName)
mcSensorTypes.head()

In [ ]:
mcSensorPositions = get_sensor_positions(iFileName)
mcSensorPositions.head()

# Kripton Maps - from True Kripton Positions

In [ ]:
def get_evt_true_positions(mcParts: pd.DataFrame
                          )      -> pd.DataFrame:
    
    # Getting the true position of the events
    # From the initial vertex of the first particle that is always primary
    evt_truePos = mcParts.loc[pd.IndexSlice[:, 1], ['initial_x', 'initial_y', 'initial_z']]
    
    # Removing the 'particle_id' column
    evt_truePos = pd.DataFrame(evt_truePos.values,
                               index=evt_truePos.index.droplevel(1),
                               columns=evt_truePos.columns)
    
    return evt_truePos

In [ ]:
# Reading all the data
mcParts                = pd.DataFrame()
PMTs_response          = pd.DataFrame()
FIBER_SENSORs_response = pd.DataFrame()

for iFileName in iFileNames:
    mcParts                = mcParts.append(
        load_mcparticles_df(iFileName))
    PMTs_response          = PMTs_response.append(
        load_mcsensor_response_df(iFileName, sns_name='PmtR11410'))
    FIBER_SENSORs_response = FIBER_SENSORs_response.append(
        load_mcsensor_response_df(iFileName, sns_name='FIBER_SENSOR'))

In [ ]:
kripton_df = get_evt_true_positions(mcParts)

In [ ]:
kripton_df['PMTs']          = PMTs_response         .groupby('event_id').charge.sum()
kripton_df['FIBER_SENSORs'] = FIBER_SENSORs_response.groupby('event_id').charge.sum()

In [ ]:
kripton_df.head()

# Visualizing the maps ...

In [ ]:
# TO BE IMPLEMENTED

In [ ]:
from matplotlib import cm

In [ ]:
kripton_df.plot(kind='scatter', x='initial_z', y='PMTs', color='blue')

In [ ]:
kripton_df.plot(kind='scatter',x='initial_z',y='FIBER_SENSORs',color='green')

In [ ]:
fig = plt.figure()
ax  = Axes3D(fig)

surf = ax.plot_trisurf(kripton_df.initial_x, kripton_df.initial_y, kripton_df.PMTs,
                cmap=cm.coolwarm, linewidth=0.2)
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.show()

In [ ]:
fig = plt.figure()
ax  = Axes3D(fig)

surf = ax.plot_trisurf(kripton_df.initial_x, kripton_df.initial_y, kripton_df.FIBER_SENSORs,
                cmap=cm.coolwarm, linewidth=0.2)
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.show()